In [62]:
from sklearn.model_selection import train_test_split
import torch
import torchvision
import pandas as pd
import os
import random
from torch.utils.data import DataLoader, Dataset
import json
import ast
from torchvision import transforms
from PIL import Image
from torchvision import datasets, models, transforms
import torch.nn as nn
import torch.optim as optim
import numpy as np
import time
import copy
from tqdm import tqdm
import fiftyone as fo
import fiftyone.zoo as foz
from fiftyone import ViewField as F


In [63]:
dataset_root = "EyePacs1KDataset"
dataset_csv = os.path.join(dataset_root, "dataset_preprocessed.csv")
df = pd.read_csv(dataset_csv)
df

Unnamed: 0  case_id                                         image_path  \
0              0  1470916  EyePacs1KDataset/587_587_resized_images/102114...   
1              1  1470916  EyePacs1KDataset/587_587_resized_images/102114...   
2              2  1470912  EyePacs1KDataset/587_587_resized_images/102114...   
3              3  1470912  EyePacs1KDataset/587_587_resized_images/102114...   
4              4  1470880  EyePacs1KDataset/587_587_resized_images/866830...   
...          ...      ...                                                ...   
2083        2083   559463  EyePacs1KDataset/587_587_resized_images/399421...   
2084        2084   546702  EyePacs1KDataset/587_587_resized_images/387357...   
2085        2085   546702  EyePacs1KDataset/587_587_resized_images/387357...   
2086        2086   523593  EyePacs1KDataset/587_587_resized_images/369955...   
2087        2087   523593  EyePacs1KDataset/587_587_resized_images/369955...   

     left_or_right_eye          dr_level      dme  moderate_dr_plus   vtdr  \
0                  lft             No DR  No CSME             False  False   
1                  lft             No DR  No CSME             False  False   
2                  lft             No DR  No CSME             False  False   
3                  lft             No DR  No CSME             False  False   
4                  lft             No DR  No CSME             False  False   
...                ...               ...      ...               ...    ...   
2083                rt  Proliferative DR  No CSME              True   True   
2084               lft             No DR  No CSME             False  False   
2085                rt     Moderate NPDR  No CSME              True  False   
2086               lft             No DR  No CSME             False  False   
2087                rt     Moderate NPDR  No CSME              True  False   

      dr_nodr  left_right_dr_mismatch  
0       False                   False  
1       False                   False  
2       False                   False  
3       False                   False  
4       False                   False  
...       ...                     ...  
2083     True                   False  
2084    False                    True  
2085     True                    True  
2086    False                    True  
2087     True                    True  

[2088 rows x 10 columns]

## Create custom dataset

In [66]:
samples = []
for _, row in df.iterrows():
    
    tags=[]

    # Create a tag for images that have left_right_mismatches
    if row["left_right_dr_mismatch"] == True:
        tags.append("left_right_dr_mismatch")
            
    dr_level = row["dr_level"]
    dr_nodr = str(row["dr_nodr"])    
    
    image_path = row["image_path"]
    sample = fo.Sample(filepath=image_path, tags=tags)
    sample["dr_level"] = fo.Classification(label=dr_level)
    sample["dr_nodr"] = fo.Classification(label=dr_nodr)
    sample["case_id"] = str(row["case_id"])
    samples.append(sample)
    
fo_dataset = fo.Dataset("eyepacs-1k-v2")
fo_dataset.add_samples(samples)

 100% |███████████████| 2088/2088 [850.6ms elapsed, 0s remaining, 2.5K samples/s]      


['63ee96d49d66fdd46d30efd9',
 '63ee96d49d66fdd46d30efda',
 '63ee96d49d66fdd46d30efdb',
 '63ee96d49d66fdd46d30efdc',
 '63ee96d49d66fdd46d30efdd',
 '63ee96d49d66fdd46d30efde',
 '63ee96d49d66fdd46d30efdf',
 '63ee96d49d66fdd46d30efe0',
 '63ee96d49d66fdd46d30efe1',
 '63ee96d49d66fdd46d30efe2',
 '63ee96d49d66fdd46d30efe3',
 '63ee96d49d66fdd46d30efe4',
 '63ee96d49d66fdd46d30efe5',
 '63ee96d49d66fdd46d30efe6',
 '63ee96d49d66fdd46d30efe7',
 '63ee96d49d66fdd46d30efe8',
 '63ee96d49d66fdd46d30efe9',
 '63ee96d49d66fdd46d30efea',
 '63ee96d49d66fdd46d30efeb',
 '63ee96d49d66fdd46d30efec',
 '63ee96d49d66fdd46d30efed',
 '63ee96d49d66fdd46d30efee',
 '63ee96d49d66fdd46d30efef',
 '63ee96d49d66fdd46d30eff0',
 '63ee96d49d66fdd46d30eff1',
 '63ee96d49d66fdd46d30eff2',
 '63ee96d49d66fdd46d30eff3',
 '63ee96d49d66fdd46d30eff4',
 '63ee96d49d66fdd46d30eff5',
 '63ee96d49d66fdd46d30eff6',
 '63ee96d49d66fdd46d30eff7',
 '63ee96d49d66fdd46d30eff8',
 '63ee96d49d66fdd46d30eff9',
 '63ee96d49d66fdd46d30effa',
 '63ee96d49d66

In [ ]:
sample = fo.Sample(filepath="foo")
help(fo_dataset.tag_samples)

## Launch FiftyOne app and create session

In [67]:
session = fo.launch_app(fo_dataset, auto=False)
session.open_tab()

Session launched. Run `session.show()` to open the App in a cell output.


<IPython.core.display.Javascript object>

In [ ]:
session.dataset = fo_dataset

In [ ]:
type(session.selected[0])

In [65]:
fo_dataset.delete()

In [70]:
session.open_tab()

<IPython.core.display.Javascript object>

In [69]:
session.show()

## Create a view that filters by tag

In [ ]:
test_view = fo_dataset.filter_field("tags", F().contains("left_right_dr_mismatch"))
session.view = test_view

In [ ]:
test_view = fo_dataset.filter_field("case_id", F() == "1453748")
session.view = test_view